In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 

In [2]:
dataset_path = r'C:\Users\reemk\aiincbs\MachineLearningCVE'

In [3]:
#appending all csv files into 1 dataframe
csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]

df_list = []
for filename in csv_files:
    file_path = os.path.join(dataset_path, filename)
    print(f"Loading: {filename}")
    df = pd.read_csv(file_path, low_memory=False)
    df_list.append(df)

df_full = pd.concat(df_list, ignore_index=True)
print(f"\n Combined shape: {df_full.shape}")


Loading: Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Loading: Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Loading: Friday-WorkingHours-Morning.pcap_ISCX.csv
Loading: Monday-WorkingHours.pcap_ISCX.csv
Loading: Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Loading: Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
Loading: Tuesday-WorkingHours.pcap_ISCX.csv
Loading: Wednesday-workingHours.pcap_ISCX.csv

 Combined shape: (2830743, 79)


<h1>Data Exploration</h1>

In [4]:
df_full.columns.unique()

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

<b>Removing Wrapping Spaces, extra spaces, etc</b>

In [5]:
df_full.columns = (
    df_full.columns
    .str.strip()                 
    .str.lower()                 
    .str.replace(' ', '_')      
    .str.replace('-', '_')       
)

In [6]:
df_full.columns

Index(['destination_port', 'flow_duration', 'total_fwd_packets',
       'total_backward_packets', 'total_length_of_fwd_packets',
       'total_length_of_bwd_packets', 'fwd_packet_length_max',
       'fwd_packet_length_min', 'fwd_packet_length_mean',
       'fwd_packet_length_std', 'bwd_packet_length_max',
       'bwd_packet_length_min', 'bwd_packet_length_mean',
       'bwd_packet_length_std', 'flow_bytes/s', 'flow_packets/s',
       'flow_iat_mean', 'flow_iat_std', 'flow_iat_max', 'flow_iat_min',
       'fwd_iat_total', 'fwd_iat_mean', 'fwd_iat_std', 'fwd_iat_max',
       'fwd_iat_min', 'bwd_iat_total', 'bwd_iat_mean', 'bwd_iat_std',
       'bwd_iat_max', 'bwd_iat_min', 'fwd_psh_flags', 'bwd_psh_flags',
       'fwd_urg_flags', 'bwd_urg_flags', 'fwd_header_length',
       'bwd_header_length', 'fwd_packets/s', 'bwd_packets/s',
       'min_packet_length', 'max_packet_length', 'packet_length_mean',
       'packet_length_std', 'packet_length_variance', 'fin_flag_count',
       'syn_flag_co

<b>Checking for missing or infinite values</b>

In [7]:
missing_values = df_full.isnull().sum()
print(missing_values[missing_values > 0])

infinite_values = df_full.isin([np.inf, -np.inf]).sum()
print(infinite_values[infinite_values > 0])


flow_bytes/s    1358
dtype: int64
flow_bytes/s      1509
flow_packets/s    2867
dtype: int64


<b>it seems that classes flow_bytes/s and flow_packets/s got 0s and inf values, dropping them</b>

In [8]:
df_full= df_full.replace([np.inf, -np.inf], np.nan, inplace=True)
df_full= df_full.dropna(subset=['flow_bytes/s', 'flow_packets/s'], inplace=True)


<b>Dropping all duplicates in records</b>

In [11]:
#view duplicate rows (across all columns)
num_duplicates = df_full.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

duplicate_percentage = (num_duplicates / len(df_full)) * 100
print(f"Duplicate rows: {num_duplicates} out of {len(df_full)} ({duplicate_percentage:.2f}%)")

Number of duplicate rows: 307078
Duplicate rows: 307078 out of 2827876 (10.86%)


<b>since its 10%, relatively low %, so ill choose to drop duplicate records, they wont add value to model training anyway</b>

In [12]:
df_full= df_full.drop_duplicates(inplace=True)